In [241]:
from tabulate import tabulate
import numpy as np
import random

n_periods: int = 8

team_members: list[str] = [
    "Benjamin",
    "Vik",
    "Vigo",
    "Can",
    "Njabulo",
    "Lars",
    "Sebas",
    "Kees",
    "Hamza",
]

excl_team: list[str] = []

tasks_base = [
    "Keeper",
    "Aanvaller",
    "Middenvelder 1",
    "Middenvelder 2",
    "Verdediger 1",
    "Verdediger 2",
]

headers = [f"Periode {i + 1}" for i in range(n_periods)]

keepers: list[str] = ["Hamza", "Vik", "Vigo", "Njabulo"]
keeper_idx = [team_members.index(keeper) for keeper in keepers]
random.shuffle(keeper_idx)

team: list[str] = [i for i in team_members if i not in excl_team]
team_idx = [team_members.index(member) for member in team]
random.shuffle(team_idx)

n_team: int = len(team)  # Number of team members
n_tasks: int = len(tasks_base)  # Number of team positions
n_subs: int = n_team - n_tasks  # The number of subs

tasks_sub = [f"Wissel {i}" for i in range(1, n_subs + 1)]
tasks = tasks_base + tasks_sub

# Create the matrix where rows are n team members playing and columns the
# n of periods
matrix: np.array = np.full((n_team, n_periods), -1).astype(int)

In [242]:
# Set the keepers
matrix[0, np.arange(len(keeper_idx)) * 2] = keeper_idx
matrix[0, np.arange(len(keeper_idx)) * 2 + 1] = keeper_idx

matrix

array([[ 4,  4,  8,  8,  1,  1,  2,  2],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1]])

In [243]:
# Set the substitutes for each period
sub_list: list = team_idx * (n_subs + 1)

for i in range(n_periods):
    for j in range(1, n_subs + 1):
        # t represents the first team member id from the list    
        t: int = sub_list[0]

        # Check if the team member is already in the column
        if np.isin(t, matrix[:,i]):
            # Take the next element in the list given t is Keeper    
            t = sub_list[1]
            # Check if team member was sub in previous period, if yes take next
            if np.isin(t, matrix[-n_subs:,i-1]):
                t = sub_list.pop(2)
            else:
                t = sub_list.pop(1)
            matrix[j * -1, i] = t
        else:
            sub_list.remove(t)
            matrix[j * -1, i] = t        

matrix

array([[ 4,  4,  8,  8,  1,  1,  2,  2],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [ 1,  8,  6,  3,  8,  0,  3,  8],
       [ 3,  7,  5,  4,  7,  6,  4,  7],
       [ 0,  2,  4,  0,  2,  5,  1,  1]])

In [244]:
# Aanvallers for each period
for i in range(n_periods):

    if (i+1) % 2 == 0:
        print(i+1)
        # We are substituting in the period
        # Get the previous attacker and check if he is substitute
        p_attack = matrix[1,i-1] # get attacker from previous period
        if p_attack in matrix[:,1]:
            print(f"the current attacker {team_members[p_attack]}  is now sub in period {i+1}")
            loc_sub = np.where(matrix[:,1] == p_attack)[0][0]
            matrix[1,i] = matrix[loc_sub, i-1]
        else:
            matrix[1,i] = matrix[1, i-1]
    else:
        unique_values, counts = np.unique(np.append(matrix[1,:], team_idx), 
                                        return_counts=True)

        # Sort the unique values and their counts
        sorted_indices = np.argsort(counts)
        sorted_values = unique_values[sorted_indices]
   
        val = np.array([el for el in sorted_values if el not in matrix[:,i]])

        matrix[1,i] = val[0]
    

2
the current attacker Vigo  is now sub in period 2
8
8
4
6
8


In [245]:
# Replace the index with the name of team members
matrix_str = [[team_members[matrix[i][j]] for j in range(n_periods)] for i in range(n_team)]

# Add tasks row
[row.insert(0, task) for task, row in zip(tasks, matrix_str)]

print(tabulate(matrix_str, headers=headers, tablefmt="grid"))

+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                | Periode 1   | Periode 2   | Periode 3   | Periode 4   | Periode 5   | Periode 6   | Periode 7   | Periode 8   |
+================+=============+=============+=============+=============+=============+=============+=============+=============+
| Keeper         | Njabulo     | Njabulo     | Hamza       | Hamza       | Vik         | Vik         | Vigo        | Vigo        |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Aanvaller      | Vigo        | Benjamin    | Vik         | Vik         | Sebas       | Sebas       | Lars        | Lars        |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Middenvelder 1 | Hamza       | Hamza       | Hamza       | Hamza       | Hamza   

In [246]:

unique_values, counts = np.unique(matrix[-n_subs:, :], return_counts=True)

# Combine the unique values and their counts into a dictionary for an overview
for value, count in zip(unique_values, counts):
    print(f"Value {team_members[value]}: {count} occurrences")




Value Benjamin: 3 occurrences
Value Vik: 3 occurrences
Value Vigo: 2 occurrences
Value Can: 3 occurrences
Value Njabulo: 3 occurrences
Value Lars: 2 occurrences
Value Sebas: 2 occurrences
Value Kees: 3 occurrences
Value Hamza: 3 occurrences


In [247]:
# Replace the index with the name of team members
matrix_str = [[team_members[matrix[i][j]] for j in range(n_periods)] for i in range(n_team)]

# Add tasks row
[row.insert(0, task) for task, row in zip(tasks, matrix_str)]

print(tabulate(matrix_str, headers=headers, tablefmt="grid"))

+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                | Periode 1   | Periode 2   | Periode 3   | Periode 4   | Periode 5   | Periode 6   | Periode 7   | Periode 8   |
+================+=============+=============+=============+=============+=============+=============+=============+=============+
| Keeper         | Njabulo     | Njabulo     | Hamza       | Hamza       | Vik         | Vik         | Vigo        | Vigo        |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Aanvaller      | Vigo        | Benjamin    | Vik         | Vik         | Sebas       | Sebas       | Lars        | Lars        |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Middenvelder 1 | Hamza       | Hamza       | Hamza       | Hamza       | Hamza   